## Master's Project: Application of Deep Learning for Solar Panel Detection in Satellite Imagery

One of the current global trends is the transition towards renewable energy sources, which helps to mitigate the effects of climate change, reduces environmental pollution and opens new prospects for energy policy reforms. Solar photovoltaic (PV) installations have become key contributors to renewable energy development, driven by the decreasing cost of producing PV cells and the increasing electrification of the energy system.

The scale of PV deployment ranges from residential rooftop installations to large industrial solar farms. Tracking the geographic locations of existing PV plants is essential for infrastructure development, statistical insights and project planning. However, in many countries, the available data is highly decentralized due to the wide scale of PV
utilization and the diversity of forms of solar plants. Not all of actual power generation from solar is accurately recorded in governmental, local or commercial databases. The lack of reliable information poses significant challenges for many participants in the energy market, primarily for network operators, project developers, policymakers and cell manufacturers. Therefore, an accurate and comprehensive database of solar panel locations and generation capacities would provide better understanding of demographic, geographic and regional trends.

**Project aim:** evaluate different deep learning network configurations in detecting solar panel installations in satellite imagery and estimate their generation capacity

**Project objectives:**

- Collect satellite imagery data and solar panel annotations
- Generate binary masks highlighting solar panels 
- Filter, split and augment the data 
- Train fully convolutional network architectures (U-Net, FPN and PSPNet) with different encoder backbones (EfficientNet-B5, ResNet-50, MiT-B3)
- Evaluate the models' performance 
- Estimate the solar energy generation capacity of the identified PV panels

In [ ]:
# Define the project folder path
folder_path = f"G:/Meine Ablage/MS Thesis/solar_panel_segmentation" # specify your own

In [ ]:
# Install necessary libraries
pip install -r requirements.txt --index-url https://download.pytorch.org/whl/cu1211

### Import libraries

In [ ]:
# Standard library
import gc
import json
import os
import random
import shutil
from glob import glob
from io import BytesIO

# Data handling
import numpy as np
import pandas as pd

# GIS & Remote Sensing
import geopandas as gpd
import rasterio
from rasterio.features import rasterize
from shapely.geometry import box
from owslib.wms import WebMapService

# Image processing
import cv2
from PIL import Image
import matplotlib.pyplot as plt
import plotly.graph_objects as go


# Machine Learning & Deep Learning
import albumentations as A
from albumentations.pytorch import ToTensorV2
import segmentation_models_pytorch as smp
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from sklearn.model_selection import train_test_split

# Metrics & Evaluation
from sklearn.metrics import (
    precision_score,
    recall_score,
    accuracy_score,
    jaccard_score,
    f1_score
)

# Utilities
from tqdm import tqdm
from natsort import natsorted
import requests

# Energy modeling
import PySAM.Pvwattsv8 as pvwatts
import pvlib

### Data input

- [German state boundaries](https://gdz.bkg.bund.de/index.php/default/verwaltungsgebiete-1-250-000-stand-01-01-vg250-01-01.html)
- [Solar panel polygons](https://www.mdpi.com/2306-5729/7/9/128#B8-data-07-00128)
- [WMS satellite imagery](https://isk.geobasis-bb.de/mapproxy/dop20c/service/wms)

All the input data, trained models, JSON files with losses and evaluation metrics can be found in [**Google Drive**](https://drive.google.com/drive/folders/10p3SCaN2at0BQcw9AK6mUX1jUAQTx9J_?usp=sharing)

In [ ]:
# Data of state boundaties in Germany
# https://gdz.bkg.bund.de/index.php/default/verwaltungsgebiete-1-250-000-stand-01-01-vg250-01-01.html

states = gpd.read_file(f"{folder_path}/VG250_LAN.shp")

# Filter Brandenburg
bb = states[states['GEN'] == 'Brandenburg']
bb = bb.to_crs("EPSG:3857")

In [ ]:
# Solar panels in Brandenburg

# Metadata: https://www.mdpi.com/2306-5729/7/9/128#B8-data-07-00128
# Download: https://zenodo.org/records/8188601

solar = gpd.read_file(f"{folder_path}/Solarenergy_Polygons_V20230420.geojson")
solar = solar.to_crs("EPSG:3857")

# Select only panels within Brandenburg
bb_solar = solar[solar.intersects(bb.unary_union)].reset_index(drop=True)

In [ ]:
# Save file
bb_solar.to_file(os.path.join(folder_path, f"solar_brandenburg.gpkg"), driver="GPKG")

In [ ]:
# Create tile geometries
# Get bounds of study area
xmin, ymin, xmax, ymax = bb.total_bounds

# Generate tiles
tile_size = 512 # in meters
tiles = []
for x in range(int(xmin), int(xmax), tile_size):
    for y in range(int(ymin), int(ymax), tile_size):
        tile_geom = box(x, y, x + tile_size, y + tile_size)
        tiles.append(tile_geom)

tiles_gdf = gpd.GeoDataFrame(geometry=tiles, crs="EPSG:3857")

In [ ]:
# Keep tiles that intersect with Brandenburg's bounds
tiles_gdf = tiles_gdf[tiles_gdf.intersects(bb.unary_union)]

In [ ]:
# Saving the tiles
tiles_gdf = tiles_gdf.reset_index(drop=True)
tiles_gdf.to_file(os.path.join(folder_path, f"tiles_brandenburg.gpkg"), driver="GPKG")

In [ ]:
# Read the file
tiles_gdf = gpd.read_file(f"{folder_path}/tiles_brandenburg.gpkg")

In [ ]:
# Download satellite image tiles

# Extract tiles from WMS
wms_url = "https://isk.geobasis-bb.de/mapproxy/dop20c/service/wms"
layer_name = "bebb_dop20c"
tile_size = 512 # meters
output_dir = os.path.join(folder_path, "images")
os.makedirs(output_dir, exist_ok=True)

# Connect to the WMS
wms = WebMapService(wms_url, version="1.3.0")

# Request and save WMS images for matched tiles
for idx, row in tiles_gdf[:].iterrows():
    minx, miny, maxx, maxy = row.geometry.bounds

    bbox = f"{minx},{miny},{maxx},{maxy}"

    params = {
        "SERVICE": "WMS",
        "VERSION": "1.3.0",
        "REQUEST": "GetMap",
        "LAYERS": layer_name,
        "STYLES": "",
        "CRS": "EPSG:3857",
        "BBOX": bbox,
        "WIDTH": tile_size,
        "HEIGHT": tile_size,
        "FORMAT": "image/jpeg"
    }

    response = requests.get(wms_url, params=params)
    if response.status_code == 200:
        image = Image.open(BytesIO(response.content))
        image.save(os.path.join(output_dir, f"tile_{idx}.png"))
    print(f"Saved tile_{idx}.png")

### Mask generation

In [ ]:
# Create a mask for each tile
output_dir = f"{folder_path}/masks"
os.makedirs(output_dir, exist_ok=True)

for idx, row in tqdm(tiles_gdf.iterrows(), total=len(tiles_gdf)):
    bounds = row.geometry.bounds
    tile_geom = box(*bounds)

    transform = rasterio.transform.from_bounds(*bounds, 512, 512)
    panels_in_tile = bb_solar[bb_solar.intersects(tile_geom)]

    mask = rasterize(
        [(geom, 1) for geom in panels_in_tile.geometry],
        out_shape=(512, 512),
        transform=transform,
        fill=0,
        dtype=np.uint8
    )

    # Save mask as PNG
    mask_path = os.path.join(output_dir, f"mask_{idx}.png")
    Image.fromarray(mask * 255).save(mask_path)

### Data filtering

In [ ]:
# Filter all masks that contain panels plus an equal number of those without panels

mask_dir = f"{folder_path}/masks"
image_dir = f"{folder_path}/images"

output_mask_dir = os.path.join(folder_path, "filtered_masks")
output_image_dir = os.path.join(folder_path, "filtered_images")
os.makedirs(output_mask_dir, exist_ok=True)
os.makedirs(output_image_dir, exist_ok=True)

# Separate lists for masks with and without panels
has_panel = []
no_panel = []

# Categorize masks
for filename in os.listdir(mask_dir):
    if filename.endswith(".png"):
        mask_path = os.path.join(mask_dir, filename)
        mask = np.array(Image.open(mask_path))

        if mask.max() > 0:
            has_panel.append(filename)
        else:
            no_panel.append(filename)

# Sample equal number of no-panel images
random.seed(42)
no_panel_sample = random.sample(no_panel, len(has_panel))

# Combine and copy selected masks/images
selected_files = has_panel + no_panel_sample

for fname in selected_files:
    shutil.copy(os.path.join(mask_dir, fname), os.path.join(output_mask_dir, fname))
    shutil.copy(os.path.join(image_dir, fname.replace("mask_", "tile_")), os.path.join(output_image_dir, fname.replace("mask_", "tile_")))

print(f"Copied {len(has_panel)} masks with panels and {len(no_panel_sample)} without panels.")

### Data split

In [ ]:
# Define paths to the images and masks
image_paths = natsorted(glob(f"{folder_path}/filtered_images/*.png"))
mask_paths = natsorted(glob(f"{folder_path}/filtered_masks/*.png"))

# Keep only base names
image_paths = [os.path.basename(p) for p in image_paths]
mask_paths = [os.path.basename(p) for p in mask_paths]

In [ ]:
# Split the data into train / val / test sets
'''
Train: 70% — used to train the model

Validation: 15% — used to tune hyperparameters and monitor overfitting

Test: 15% — used to evaluate final model performance
'''

# Split into train+val and test
img_trainval, img_test, mask_trainval, mask_test = train_test_split(
    image_paths, mask_paths, test_size=0.15, random_state=42
)

# Split train+val into train and val
img_train, img_val, mask_train, mask_val = train_test_split(
    img_trainval, mask_trainval, test_size=0.176, random_state=42  # 0.176 ≈ 15% / 85%
)

In [ ]:
# Create folders for training, validation and test sets
base_dir = f"{folder_path}/solar_detection/datasets"
splits = ['train', 'val', 'test']
for split in splits:
    os.makedirs(os.path.join(base_dir, split, "images"), exist_ok=True)
    os.makedirs(os.path.join(base_dir, split, "masks"), exist_ok=True)

In [ ]:
# Copy files for each split
base_dir = f"{folder_path}/solar_detection/datasets"
image_dir = f"{folder_path}/solar_detection/filtered_images"
mask_dir = f"{folder_path}/solar_detection/filtered_masks"

def copy_split(images, masks, split):
    for img, msk in zip(images, masks):
        shutil.copy(os.path.join(image_dir, img), os.path.join(base_dir, split, "images", img))
        shutil.copy(os.path.join(mask_dir, msk), os.path.join(base_dir, split, "masks", msk))

copy_split(img_train, mask_train, "train")
copy_split(img_val, mask_val, "val")
copy_split(img_test, mask_test, "test")

In [ ]:
# Read the folders
base_dir = f"{folder_path}/solar_detection/datasets"
img_train  = natsorted(glob(f"{base_dir}/train/images/*.png"))
mask_train = natsorted(glob(f"{base_dir}/train/masks/*.png"))

img_val  = natsorted(glob(f"{base_dir}/val/images/*.png"))
mask_val = natsorted(glob(f"{base_dir}/val/masks/*.png"))

img_test  = natsorted(glob(f"{base_dir}/test/images/*.png"))
mask_test = natsorted(glob(f"{base_dir}/test/masks/*.png"))

### Data augmentation

In [ ]:
# Data augmentation: horizontal flipping, image rotation, shift-scaling transformations and normalization

train_transform = A.Compose([
    A.HorizontalFlip(p=0.3), 
    A.RandomRotate90(p=0.3),
    A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.2, rotate_limit=20, p=0.3),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),  # ImageNet stats
    ToTensorV2()
])

val_transform = A.Compose([
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])

test_transform = A.Compose([
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])

### Network configuration

In [ ]:
# Dataset class
class SolarPanelDataset(Dataset):
    def __init__(self, image_paths, mask_paths, transform=None):
        self.image_paths = image_paths
        self.mask_paths = mask_paths
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image = np.array(Image.open(self.image_paths[idx]).convert("RGB").resize((512, 512)))
        mask = np.array(Image.open(self.mask_paths[idx]).convert("L").resize((512, 512)))
        mask = (mask > 0.5).astype(np.float32)

        if self.transform:
            augmented = self.transform(image=image, mask=mask)
            image = augmented["image"]
            mask = augmented["mask"].unsqueeze(0)
        else:
            image = torch.tensor(image / 255.0, dtype=torch.float).permute(2, 0, 1)
            mask = torch.tensor(mask, dtype=torch.float).unsqueeze(0)

        return image, mask

In [ ]:
# Create Dataset objects and DataLoaders
train_dataset = SolarPanelDataset(img_train, mask_train, transform=train_transform)
val_dataset = SolarPanelDataset(img_val, mask_val, transform=val_transform)
test_dataset = SolarPanelDataset(img_test, mask_test, transform=test_transform)

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, num_workers=4, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False, num_workers=4, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False, num_workers=4, pin_memory=True)

In [ ]:
# Define model architectures with different backbones
device = torch.device("cuda")

# Model architectures
architectures = {
    "unet": smp.Unet, # U-Net
    "fpn": smp.FPN, # Feature Pyramid Network
    "pspnet": smp.PSPNet # Pyramid Scene Parsing Network
}

# Encoder backbones
backbones = ['mit_b3', 'resnet50', 'efficientnet-b5'] # MiT-B3, ResNet-50, EfficientNet-B5

models = {}
optimizers = {}

for arch_name, arch_class in architectures.items():
    for encoder in backbones:
        model_name = f"{arch_name}_{encoder}"
        model = arch_class(
            encoder_name=encoder,
            encoder_weights="imagenet",
            in_channels=3,
            classes=1,
            activation=None
        ).to(device)
        models[model_name] = model
        optimizers[model_name] = torch.optim.Adam(model.parameters(), lr=1e-3)

In [ ]:
# Calculate a balanced weight for the positive class in Binary Cross Entropy Loss
mask_paths = natsorted(glob(f"{folder_path}/solar_detection/filtered_masks/*.png"))

total_pos = 0
total_neg = 0

for path in tqdm(mask_paths):
    mask = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    mask = (mask > 127).astype(np.uint8)

    pos = np.sum(mask == 1)
    neg = np.sum(mask == 0)

    total_pos += pos
    total_neg += neg

pos_weight = total_neg / total_pos
print("Calculated pos_weight:", pos_weight)

In [ ]:
# Define loss functions: Dice loss and Binary Cross Entropy loss
dice_loss_fn = smp.losses.DiceLoss(mode='binary')

# Define BCE loss with positive class weight
pos_weight = torch.tensor([pos_weight]).to(device)
bce_loss_fn = torch.nn.BCEWithLogitsLoss(pos_weight=pos_weight)

In [ ]:
# Model training and validating
num_epochs = 5
history = {}

for model_name, model in models.items():
    optimizer = optimizers[model_name]
    print(f"Training model: {model_name}")

    history[model_name] = {
        "train_loss": [],
        "val_loss": []
    }

    for epoch in range(num_epochs):
        print(f"Epoch {epoch+1}/{num_epochs}")

        # Training
        model.train()
        running_train_loss = 0.0

        train_bar = tqdm(train_loader, desc=f"{model_name} - Training", leave=False)
        for images, masks in train_bar:
            images = images.to(device)
            masks = masks.to(device)

            optimizer.zero_grad()
            outputs = model(images)

            dice_loss = dice_loss_fn(torch.sigmoid(outputs), masks)
            bce_loss = bce_loss_fn(outputs, masks)
            loss = dice_loss + bce_loss

            loss.backward()
            optimizer.step()

            running_train_loss += loss.item()
            train_bar.set_postfix(loss=loss.item())

        avg_train_loss = running_train_loss / len(train_loader)
        history[model_name]["train_loss"].append(avg_train_loss)

        # Validation
        model.eval()
        running_val_loss = 0.0

        val_bar = tqdm(val_loader, desc=f"{model_name} - Validation", leave=False)
        with torch.no_grad():
            for images, masks in val_bar:
                images = images.to(device)
                masks = masks.to(device)

                outputs = model(images)

                dice_loss = dice_loss_fn(torch.sigmoid(outputs), masks)
                bce_loss = bce_loss_fn(outputs, masks)
                loss = dice_loss + bce_loss

                running_val_loss += loss.item()
                val_bar.set_postfix(loss=loss.item())

        avg_val_loss = running_val_loss / len(val_loader)
        history[model_name]["val_loss"].append(avg_val_loss)

        print(f"Epoch {epoch+1}: Train Loss = {avg_train_loss:.4f}, Val Loss = {avg_val_loss:.4f}")

    # Save model weights
    save_dir = f"{folder_path}/trained_models"
    os.makedirs(save_dir, exist_ok=True)

    model_path = os.path.join(save_dir, f"{model_name}.pth")
    torch.save(model.state_dict(), model_path)
    del model
    del optimizer
    torch.cuda.empty_cache()
    gc.collect()

    # Save training history
    history_path = os.path.join(save_dir, f"{model_name}_loss.json")
    with open(history_path, "w") as f:
        json.dump(history[model_name], f)

    print(f"Saved model and history: {model_name}")

In [ ]:
# Device setup
device = torch.device("cuda")

# Set model configuration
in_channels = 3
classes = 1
models_dir = f"{folder_path}/trained_models"
results = {}

# Loop through all saved models
for filename in os.listdir(models_dir):
    if filename.endswith(".pth") and filename.startswith('pspnet'):
        model_name = filename.replace(".pth", "")
        print(f"Evaluating model: {model_name}")

        # Parse model architecture and encoder name
        arch, encoder = model_name.split("_", 1)
        arch_class = smp.PSPNet

        # Load model
        model = arch_class(
            encoder_name=encoder,
            encoder_weights=None,
            in_channels=in_channels,
            classes=classes,
            activation=None
        ).to(device)
        model.load_state_dict(torch.load(os.path.join(models_dir, filename), map_location=device))
        model.eval()

        # Initialize storage for all predictions and ground truths
        y_true, y_pred = [], []

        # Run evaluation on the test set
        with torch.no_grad():
            test_bar = tqdm(test_loader, desc=f"{model_name} - Testing", leave=False)
            for images, masks in test_bar:
                images = images.to(device)
                masks = masks.to(device)

                outputs = model(images)
                probs = torch.sigmoid(outputs)
                preds = (probs > 0.5).float()

                y_true.extend(masks.cpu().numpy().reshape(-1))
                y_pred.extend(preds.cpu().numpy().reshape(-1))

        # Compute metrics
        metrics = {
            "Accuracy": accuracy_score(y_true, y_pred),
            "Precision": precision_score(y_true, y_pred, zero_division=0),
            "Recall": recall_score(y_true, y_pred, zero_division=0),
            "Dice Coefficient": f1_score(y_true, y_pred, zero_division=0),
            "IoU": jaccard_score(y_true, y_pred, zero_division=0)
        }
        results[model_name] = metrics

        # Save metrics to a JSON file
        metrics_path = os.path.join(models_dir, f"{model_name}_metrics.json")
        with open(metrics_path, "w") as f:
          json.dump(metrics, f, indent=4)

# Print all metrics
for name, metrics in results.items():
    print(f"Metrics for {name}:")
    for k, v in metrics.items():
        print(f"{k}: {v:.4f}")

### Performance evaluation

In [ ]:
# Model performance evaluation

# Path to the directory containing the metric JSON files
json_dir = f"{folder_path}/trained_models/evaluation"

# Collect all JSON files ending with "_metrics.json"
json_files = [f for f in os.listdir(json_dir) if f.endswith("_metrics.json")]

# Container for parsed data
rows = []

# Loop through each file and extract the relevant info
for filename in json_files:
    filepath = os.path.join(json_dir, filename)

    with open(filepath, 'r') as f:
        metrics = json.load(f)

    # Parse architecture and backbone from filename
    name_parts = filename.replace("_metrics.json", "").split("_")
    architecture = name_parts[0].upper()
    backbone = "_".join(name_parts[1:]).title()

    # Extract evaluation metrics
    metric_items = list(metrics.items())

    row = {
        "Architecture": architecture,
        "Backbone": backbone
    }
    row.update(metric_items)
    rows.append(row)

# Create a DataFrame
df = pd.DataFrame(rows)

# Convert metric values to percentage format (2 decimal places)
metric_columns = [col for col in df.columns if col not in ["Architecture", "Backbone"]]
df[metric_columns] = df[metric_columns].applymap(lambda x: f"{x * 100:.2f}%" if isinstance(x, (int, float)) else x)

# Save or display the table
print(df.to_string(index=False))  # Show in console  # Show in console

In [ ]:
# Show example predictions and compare with the ground truth mask

# Load a sample image and mask
image_path = f"{folder_path}/datasets/test/images/tile_107630.png"
mask_path = f"{folder_path}/datasets/test/masks/mask_107630.png"
model_path = f"{folder_path}/trained_models/models/unet_efficientnet-b5.pth"

# Load model: U-Net with EfficientNet-B5 backbone as the best-performing model
model = smp.Unet(
    encoder_name="efficientnet-b5",
    encoder_weights='imagenet',
    in_channels=3,
    classes=1,
    activation=None
)
model.load_state_dict(torch.load(model_path, map_location="cpu"))
model.eval()

# Preprocessing
test_transform = A.Compose([
    A.Resize(512, 512),
    A.Normalize(mean=(0.485, 0.456, 0.406),
                std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])

# Load and prepare input image
image = Image.open(image_path).convert("RGB")
image_np = np.array(image)
transformed = test_transform(image=image_np)
input_tensor = transformed['image'].unsqueeze(0)

# Load and resize mask (for comparison only, not normalized)
mask = Image.open(mask_path).convert("L").resize((512, 512))
mask_np = np.array(mask)

# Predict
with torch.no_grad():
    output = model(input_tensor)[0, 0]
    pred_mask = (torch.sigmoid(output) > 0.5).float().numpy()

# Plot results
plt.figure(figsize=(12, 4))

plt.subplot(1, 3, 1)
plt.imshow(image)
plt.axis("off")

plt.subplot(1, 3, 2)
plt.imshow(mask_np, cmap="gray")
plt.axis("off")

plt.subplot(1, 3, 3)
plt.imshow(pred_mask, cmap="gray")
plt.axis("off")

plt.tight_layout()
plt.show()

In [ ]:
# Plot train and validation losses by epoch for U-Net, FCN and PSPNet

# Paths to the JSON files with losses
loss_files = {
    "U-Net + EfficientNet-B5": f"{folder_path}/trained_models/train-val loss/unet_efficientnet-b5_loss.json",
    "FPN + EfficientNet-B5": f"{folder_path}/trained_models/train-val loss/fpn_efficientnet-b5_loss.json",
    "PSPNet + EfficientNet-B5": f"{folder_path}/trained_models/train-val loss/pspnet_efficientnet-b5_loss.json"
}

# Load and plot each model
for model_name, filepath in loss_files.items():
    with open(filepath, "r") as f:
        data = json.load(f)

    epochs = list(range(1, len(data["train_loss"]) + 1))
    train_loss = data["train_loss"]
    val_loss = data["val_loss"]

    # Plot
    plt.figure(figsize=(6, 4))
    plt.plot(epochs, train_loss, label='Train', marker='o', color='royalblue')
    plt.plot(epochs, val_loss, label='Val', marker='o', color='firebrick')

    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.xticks(epochs)  
    plt.legend(loc='upper right', frameon=True)
    plt.grid(False)
    plt.tight_layout()
    plt.show()


In [ ]:
# Predict a total area covered by solar panels in Brandenburg

# Device selection
device = torch.device("cuda" )

# Paths
image_path = f"{folder_path}/filtered_images"
model_path = f"{folder_path}/trained_models/models/unet_efficientnet-b5.pth" 

# === Load model ===
model = smp.Unet(
    encoder_name="efficientnet-b5",
    encoder_weights='imagenet',
    in_channels=3,
    classes=1,
    activation=None
).to(device)
model.load_state_dict(torch.load(model_path, map_location=device))
model.eval()

# === Transform
transform = A.Compose([
    A.Resize(512, 512),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])

# === Area counter
total_area_m2 = 0

# === Inference over all images
image_files = [f for f in os.listdir(image_path)]

for filename in tqdm(image_files, desc="Predicting"):
    img_path = os.path.join(image_path, filename)

    image = Image.open(img_path).convert("RGB")
    image_np = np.array(image)
    transformed = transform(image=image_np)
    input_tensor = transformed["image"].unsqueeze(0).to(device)

    with torch.no_grad():
        output = model(input_tensor)[0, 0]
        pred_mask = (torch.sigmoid(output).cpu() > 0.5).float().numpy()

    total_area_m2 += np.sum(pred_mask)  # 1 pixel = 1 m²

# === Print total area
print(f"Total solar panel area detected in Brandenburg:")
print(f"{int(total_area_m2)} m²")

### Solar energy generation capacity estimation

In [ ]:
# Input
total_area_m2 = 57325941
average_generation_capacity = 0.2  # kW/m²
dc_ac_ratio = 1.1
system_capacity_kw = (total_area_m2 * average_generation_capacity) / 1000

# Load EPW data via pvlib
epw_path = f"{folder_path}/tmy_52.413_13.383_2005_2023.epw"
data, meta = pvlib.iotools.read_epw(epw_path)

weather_data = {
    'year': data['year'].to_numpy(dtype=float),
    'month': data['month'].to_numpy(dtype=float),
    'day': data['day'].to_numpy(dtype=float),
    'hour': (data['hour']).to_numpy(dtype=float),
    'minute': data['minute'].to_numpy(dtype=float),

    'dn': data['dni'].to_numpy(dtype=float),
    'df': data['dhi'].to_numpy(dtype=float),
    'gh': data['ghi'].to_numpy(dtype=float),
    'drybulb': data['temp_air'].to_numpy(dtype=float),
    'wspd': data['wind_speed'].to_numpy(dtype=float),

    'tz': float(meta['TZ']),
    'lat': float(meta['latitude']),
    'lon': float(meta['longitude']),
    'elev': float(meta['altitude'])
}

# === Run PVWatts with in-memory weather ===
pv = pvwatts.default("PVWattsSingleOwner")
pv.SolarResource.solar_resource_data = weather_data

pv.SystemDesign.system_capacity = system_capacity_kw
pv.SystemDesign.dc_ac_ratio = dc_ac_ratio
pv.SystemDesign.module_type = 0
pv.SystemDesign.array_type = 1
pv.SystemDesign.tilt = 30
pv.SystemDesign.azimuth = 180

pv.execute()

# Output
annual_kwh = pv.Outputs.ac_annual
annual_mwh = annual_kwh / 1000

print(f"Estimated annual energy generation:")
print(f"{annual_kwh:,.0f} kWh / {annual_mwh:,.2f} MWh")

Original solar panel area from the dataset: 9.81 TWh/year

Identified solar panel area: 10.72 TWh/year

### Conclusion

This project investigated the application of modern deep learning networks to identify accurate locations and shapes of solar panels in satellite imagery of Brandenburg, Germany. A comprehensive framework is proposed to select a best-performing model for semantic segmentation of solar panels based on WMS satellite imagery and to estimate the solar energy generation of detected installations. 

The study presented a comparative evaluation of the performance of three fully convolutional network architectures (U-Net, FPN and PSPNet) with three different encoder backbones (EfficientNet-B5, ResNet-50, MiT-B3). Furthermore, data augmentation techniques were applied along with assigning class-specific weights to the loss function to address data imbalance. Almost all the evaluated model configurations demonstrated robust segmentation performance. U-Net with EfficientNet-B5 backbone showed the best prediction results with 85.39% IoU and 92.12% Dice, which is one of the highest scores achieved on a low- resolution satellite imagery in the literature. The model also satisfied the imposed computational constraints, rapidly reaching convergence after the first few training epochs. Reliability of a data source and accuracy of ground truth annotations proved to be the most critical factors affecting the model performance. The method for solar energy output estimation provided a practical means of generating further insights from the identified PV module objects. The results of the project contribute to the application of deep learning models to support data monitoring and validation in solar energy industry and other fields where remote sensing might be needed.